In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# `Logit` on Orders - Logistic Regression (~1h)

## Select features

🎯 Haydi `wait_time` ve `delay_vs_expected` değişkenlerinin çok `iyi/kötü review`lar üzerindeki etkisini inceleyelim.

👉 `orders` training_set’imizi kullanarak iki adet `multivariate logistic regression` çalıştıracağız:
- `logit_one` → `dim_is_one_star` tahmini için  
- `logit_five` → `dim_is_five_star` tahmini için.

 

In [14]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Dataset’inizi import edin:

In [18]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

In [16]:
print(orders)

                               order_id  wait_time  expected_wait_time  \
0      e481f51cbdc54678b7cc49136f2d6af7   8.436574           15.544063   
1      53cdb2fc8bc7dce0b6741e2150273451  13.782037           19.137766   
2      47770eb9100c2d0c44946d9cf07ec65d   9.394213           26.639711   
3      949d5b44dbf5de918fe9c16f97b45f8a  13.208750           26.188819   
4      ad21c59c0840e6cb83a9ceb5573f8159   2.873877           12.112049   
...                                 ...        ...                 ...   
97002  9c5dedf39a927c1b2549525ed64a053c   8.218009           18.587442   
97003  63943bddc261676b46f01ca7ac2f7bd8  22.193727           23.459051   
97004  83c1379a015df1e13d02aae0204711ab  24.859421           30.384225   
97005  11c177c8e97725db2631073c19f07b62  17.086424           37.105243   
97006  66dea50a8b16d9b4dee7af250b4be1a5   7.674306           25.126736   

       delay_vs_expected order_status  dim_is_five_star  dim_is_one_star  \
0                    0.0    deliver

In [17]:
type(orders)

pandas.core.frame.DataFrame

👉 Kullanmak istediğiniz feature’ları bir listede seçin:

⚠️ Data leakage yaratmadığınızdan emin olun (yani target’tan türetilmiş feature’ları seçmeyin)

💡 `wait_time` ve `delay_vs_expected` değişkenlerinin etkisini anlayabilmek için diğer feature’ların etkisini kontrol etmemiz gerekir, bu yüzden listenize ilgili olabilecek tüm feature’ları dahil edin.

In [19]:
features = [
    'wait_time',
    'delay_vs_expected',
    'price',
    'freight_value',
    'number_of_items',
    'number_of_sellers'
]

🕵🏻 Feature’larınızın `multicollinearity` durumunu `VIF index` kullanarak kontrol edin.

* Çok yüksek olmamalıdır (tercihen < 10), böylece partial regression coefficient’larına ve ilgili `p-values` değerlerine güvenebiliriz.
* Verinizi standardize etmeyi unutmayın!
    * Bir `VIF Analysis`, bir feature’ın diğer feature’lara karşı regresyonunu yaparak hesaplanır...
    * Bu yüzden herhangi bir linear regression çalıştırmadan önce feature’ların `scale etkisini kaldırmak` ve eşit öneme sahip olmalarını sağlamak istersiniz!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Standardize etme:

In [20]:
from sklearn.preprocessing import StandardScaler

# Eksik değerleri temizle
orders_clean = orders[features + ['dim_is_one_star', 'dim_is_five_star']].dropna()

# Standardize
scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(orders_clean[features]),
    columns=features,
    index=orders_clean.index
)

print("Standardizasyon tamamlandı!")

Standardizasyon tamamlandı!


👉 Olası multicollinearity durumlarını analiz etmek için VIF Analysis’inizi çalıştırın:

In [21]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF hesapla
vif_data = pd.DataFrame()
vif_data['Feature'] = features
vif_data['VIF'] = [
    variance_inflation_factor(X_scaled.values, i) 
    for i in range(len(features))
]

vif_data = vif_data.sort_values('VIF', ascending=False)
print(vif_data)

             Feature       VIF
0          wait_time  2.105359
1  delay_vs_expected  2.023235
3      freight_value  1.542023
4    number_of_items  1.343850
2              price  1.204691
5  number_of_sellers  1.093009


## Logistic Regressions

👉 İki adet `Logistic Regression` modeli fit edin:
- `logit_one` → `dim_is_one_star` tahmini için
- `logit_five` → `dim_is_five_star` tahmini için.

`Logit 1️⃣`

In [22]:
import statsmodels.api as sm

# X ve y hazırla
X = X_scaled
y_one = orders_clean['dim_is_one_star']

# Intercept ekle
X_with_const = sm.add_constant(X)

# Logistic Regression fit et
logit_one = sm.Logit(y_one, X_with_const).fit()

# Sonuçları göster
print(logit_one.summary())

Optimization terminated successfully.
         Current function value: 0.274399
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:        dim_is_one_star   No. Observations:                96353
Model:                          Logit   Df Residuals:                    96346
Method:                           MLE   Df Model:                            6
Date:                Wed, 25 Feb 2026   Pseudo R-squ.:                  0.1420
Time:                        07:54:23   Log-Likelihood:                -26439.
converged:                       True   LL-Null:                       -30814.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -2.4603      0.013   -191.577      0.000      -2.486      -2.435
wait_tim

`Logit 5️⃣`

In [23]:
# y değiştir, geri kalanı aynı
y_five = orders_clean['dim_is_five_star']

# Logistic Regression
logit_five = sm.Logit(y_five, X_with_const).fit()

# Sonuçları göster
print(logit_five.summary())

Optimization terminated successfully.
         Current function value: 0.637590
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:       dim_is_five_star   No. Observations:                96353
Model:                          Logit   Df Residuals:                    96346
Method:                           MLE   Df Model:                            6
Date:                Wed, 25 Feb 2026   Pseudo R-squ.:                 0.05689
Time:                        07:54:57   Log-Likelihood:                -61434.
converged:                       True   LL-Null:                       -65140.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.3387      0.007     47.504      0.000       0.325       0.353
wait_tim

In [24]:
# Katsayıları yan yana göster
comparison = pd.DataFrame({
    'logit_one': logit_one.params,
    'logit_five': logit_five.params,
    'p_value_one': logit_one.pvalues,
    'p_value_five': logit_five.pvalues
})

print(comparison)

                   logit_one  logit_five    p_value_one  p_value_five
const              -2.460329    0.338740   0.000000e+00  0.000000e+00
wait_time           0.616627   -0.466631   0.000000e+00  0.000000e+00
delay_vs_expected   0.317705   -0.471405   1.971985e-70  9.275012e-94
price               0.060009    0.015571   1.073384e-07  4.089453e-02
freight_value      -0.072397    0.032755   1.320623e-07  1.941171e-04
number_of_items     0.272960   -0.149696  2.712348e-145  1.094427e-72
number_of_sellers   0.175813   -0.141038  4.574699e-112  4.577304e-73


💡 Şimdi bu iki logistic regression’ın sonuçlarını analiz etme zamanı:

- Partial coefficient’ları kendi kelimelerinizle yorumlayın.
- `p-values` kullanarak istatistiksel anlamlılıklarını kontrol edin.
- Coefficient önemleri açısından `logit_one` ve `logit_five` arasında herhangi bir fark görüyor musunuz?

In [25]:
# Aşağıdaki cümlelerden doğru olanları aşağıdaki listeye kaydedin.

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more than one_star"

your_answer = [a]

🧪 __Kodunu Test Et__

In [26]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/semih/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/semih/code/Semih0799/data-logit/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_logit.py::TestLogit::test_question PASSED                           [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/logit.pickle

git commit -m 'Completed logit step'

git push origin master



<details>
    <summary>- <i>Açıklamalar ve ileri seviye kavramlar</i> -</summary>


> _Diğer tüm şeyler sabitken, `delay factor`, 1-yıldız review alma ihtimalini etkilemesinden bile daha fazla, 5-yıldızdan mahrum kalma ihtimalini artırma eğilimindedir. Muhtemelen bunun sebebi, 1-yıldız review’ların bizzat çok kötü ürünleri hedeflemesi, kötü teslimatları değil._

❗️ Ancak tamamen titiz olmak için, **iki farklı modelin coefficient’larını karşılaştırırken daha dikkatli olmamız gerekir**, çünkü **benzer popülasyonlara dayanmayabilirler**!
    Burada 2 alt popülasyonumuz var: (1-yıldız verenler ve 5-yıldız verenler) ve bunlar doğaları gereği farklı davranış kalıpları sergileyebilirler. 5-yıldız vermeye daha meyilli “mutlu insanlar”ın, “gecikme” veya “fiyat” söz konusu olduğunda, 1-yıldızı “Lucky-Luke gibi ateşleyen” “huysuz insanlara” göre daha az hassas olmaları gayet mümkün...

</details>



🏁 Tebrikler!

💾 `logit.ipynb` notebook’unuzu commit ve push etmeyi unutmayın!